<H3> Manipulação de tabelas utilizadas para filtragem baseada em conteúdo </H3>
<h6> Utilizada na etapa de Pŕe-Processamento de dados </h6>

In [1]:
import pandas as pd
df1 = pd.read_csv('./csv/movies_metadata.csv')
#removendo Id's inconsistentes 
df1 = df1.drop([19730, 29502, 29503, 35587])

#removendo atributos que não precisamos
df1 = df1.drop(columns=['adult','belongs_to_collection', 'homepage', 'imdb_id', 'original_title', 
                        'overview', 'poster_path','production_countries', 'revenue', 'spoken_languages', 
                        'status', 'tagline', 'vote_count', 'vote_average', 'video'])

#transformando dados em numéricos
df1['id'] = pd.to_numeric(df1['id'])
df1['budget'] = pd.to_numeric(df1['budget'])
df1['popularity'] = pd.to_numeric(df1['popularity'])

#apenas dropando se houver um nan
df1 = df1.dropna()
df1

FileNotFoundError: [Errno 2] File b'./csv/movies_metadata.csv' does not exist: b'./csv/movies_metadata.csv'

In [3]:
df2 = pd.read_csv('./csv/credits.csv')
df = df1.merge(df2, on='id').drop_duplicates()

In [9]:
df[['genres', 'production_companies', 'cast', 'crew']]

,genres,production_companies,cast,crew
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",[{'name': 'Twentieth Century Fox Film Corporat...,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,"[{'id': 35, 'name': 'Comedy'}]","[{'name': 'Sandollar Productions', 'id': 5842}...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...
45189,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...","[{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...","[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de..."
45190,"[{'id': 18, 'name': 'Drama'}]","[{'name': 'Sine Olivia', 'id': 19653}]","[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de..."
45191,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...","[{'name': 'American World Pictures', 'id': 6165}]","[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de..."
45192,[],"[{'name': 'Yermoliev', 'id': 88753}]","[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de..."


In [3]:
from ast import literal_eval

features = ['genres', 'production_companies', 'cast', 'crew']
for feature in features:
    df[feature] = df[feature].apply(literal_eval)
    

In [4]:
#selecting director name
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return ''

df['director'] = df['crew'].apply(get_director)

In [5]:
#get names in the case the attribute it's a list
def get_names(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        return names

features = ['genres', 'production_companies', 'cast']
for feature in features:
    df[feature] = df[feature].apply(get_names)

#making a copy of the df that not have modifications
originalDf = df[['id', 'title']]

In [6]:
df = df.drop(columns='crew')

In [7]:
#passing strings to lower case and removing the spaces between then
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    elif isinstance(x, str):
        return str.lower(x.replace(" ", ""))
    else:
        return ''

stringColumns = ['genres', 'original_language', 'production_companies', 'release_date', 'cast', 'director']

for column in stringColumns:
    df[column] = df[column].apply(clean_data)
    



In [8]:
def get_year(date):
    return date[0:4]

df['release_date'] = df['release_date'].apply(get_year)
df

,budget,genres,id,original_language,popularity,production_companies,release_date,runtime,title,cast,director
0,30000000,"[animation, comedy, family]",862,en,21.946943,[pixaranimationstudios],1995,81.0,Toy Story,"[tomhanks, timallen, donrickles, jimvarney, wa...",johnlasseter
1,65000000,"[adventure, fantasy, family]",8844,en,17.015539,"[tristarpictures, teitlerfilm, interscopecommu...",1995,104.0,Jumanji,"[robinwilliams, jonathanhyde, kirstendunst, br...",joejohnston
2,0,"[romance, comedy]",15602,en,11.712900,"[warnerbros., lancastergate]",1995,101.0,Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret, sophi...",howarddeutch
3,16000000,"[comedy, drama, romance]",31357,en,3.859495,[twentiethcenturyfoxfilmcorporation],1995,127.0,Waiting to Exhale,"[whitneyhouston, angelabassett, lorettadevine,...",forestwhitaker
4,0,[comedy],11862,en,8.387519,"[sandollarproductions, touchstonepictures]",1995,106.0,Father of the Bride Part II,"[stevemartin, dianekeaton, martinshort, kimber...",charlesshyer
...,...,...,...,...,...,...,...,...,...,...,...
45189,0,"[drama, action, romance]",30840,en,5.683753,"[westdeutscherrundfunk(wdr), workingtitlefilms...",1991,104.0,Robin Hood,"[patrickbergin, umathurman, davidmorrissey, jü...",johnirvin
45190,0,[drama],111109,tl,0.178241,[sineolivia],2011,360.0,Century of Birthing,"[angelaquino, perrydizon, hazelorencio, joelto...",lavdiaz
45191,0,"[action, drama, thriller]",67758,en,0.903007,[americanworldpictures],2003,90.0,Betrayal,"[erikaeleniak, adambaldwin, juliedupage, james...",markl.lester
45192,0,[],227506,en,0.003503,[yermoliev],1917,87.0,Satan Triumphant,"[iwanmosschuchin, nathalielissenko, pavelpavlo...",yakovprotazanov


In [9]:
#agrupando todas os metadados em string em apenas um
def join_metadata(x):
    return ' '.join(x['genres']) + ' ' + x['original_language'] + ' ' + ' '.join(x['production_companies']) + 
        ' ' + x['release_date'] + ' ' + ' '.join(x['cast']) + ' ' + x['director']

df['stringMetadata'] = df.apply(join_metadata, axis=1) #eixo das colunas

In [18]:
#saving the dataframes to a CSV file
originalDf.to_csv('./csv/contentPrettyDf.csv')
df.to_csv('./csv/contentDf.csv')

In [10]:
#transforming stringMetadata in a matrix with each column a word of the vocabulary
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(stop_words='english', min_df=1)
matrix = vect.fit_transform(df['stringMetadata'])
sdf = pd.DataFrame.sparse.from_spmatrix(matrix, columns=vect.get_feature_names(), index=df.index)

sdf

,01distribution,02entertainment,100,1000volt,100bares,100doo,100feetproductions,1019entertainment,101ststreetfilms,106,...,씨너스엔터테인먼트,영화사수박,영화사집,이디오플랜,인벤트디,조병만,진모영,타임스토리그룹,팔레트픽처스,프로덕션m
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45189,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45190,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45192,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:

#joining the number fields with the string metadata dataframe
numberDf = df[['budget', 'popularity', 'runtime']]
numberValues = numberDf.values

#TEST VALUES
testNumber = [numberDf.loc[9193].values]
testNumber


[array([ 0.      ,  6.228255, 87.      ])]

In [15]:
import scipy.sparse as sps
sps.save_npz('./csv/sparseMatrix.npz', matrix)

In [16]:
numberDf.to_csv('./csv/numberDF.csv')

In [15]:
#transforming description attributes to a sparse matrix
ssdf = sdf.to_sparse()
sMatrix = ssdf.sparse.to_coo()

sMatrix

/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: FutureWarning: DataFrame.to_sparse is deprecated and will be removed in a future version
  
/home/sid/.local/lib/python3.6/site-packages/pandas/core/frame.py:3471: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  return klass(values, index=self.index, name=items, fastpath=True)


<45112x241084 sparse matrix of type '<class 'numpy.int64'>'
	with 889277 stored elements in COOrdinate format>

In [1]:
import scipy.sparse as sps
#creating test Sparse matrix to textual attr
testMatrix = sps.coo_matrix(sdf.loc[9193].values)
print(testMatrix)

NameError: name 'sdf' is not defined

In [17]:
#calculating similarities
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
sMatrix = sMatrix.astype(float)
testMatrix = testMatrix.astype(float)
similarities = cosine_similarity(testMatrix, sMatrix)

similarities2 = cosine_similarity(testNumber, numberValues)

In [18]:
#joining similarities
resultSimilarities = []
for i in range(0, 45112):
    sim1 = similarities[0]
    sim2 = similarities2[0]
    resultSimilarities.append((7*sim1[i] + 3*sim2[i])/10)

resultSimilarities

[0.030124027407673153,
 4.843824831840453e-07,
 0.29970982678380287,
 0.032675582299411046,
 0.29999157199270216,
 0.015353060445709426,
 6.576859037125214e-07,
 0.3344786405436808,
 9.094541368239803e-07,
 6.761215166777928e-07,
 0.02592643970298273,
 0.29998548496708494,
 0.32705866317777355,
 0.02078829371037738,
 3.649481252132314e-07,
 0.017844474445564647,
 0.02750107731825054,
 7.37957938842695e-06,
 9.035617892924069e-07,
 5.163052292756121e-07,
 1.0476362784036604e-06,
 0.32805795241990393,
 7.947193386018806e-07,
 0.3717983540853796,
 0.029406607778838762,
 0.33430522944677954,
 0.027501106161959527,
 0.32307531182295124,
 0.014202021548717256,
 0.33422028202261517,
 0.3263333103710561,
 0.015157946169615782,
 0.2995203156162776,
 0.02545965187488995,
 0.3258955673714659,
 0.019054919088004725,
 0.33664475494687596,
 0.2999956448125272,
 0.029399673240955153,
 0.33115697503800046,
 0.32682845911165553,
 0.028093614437493218,
 0.033680620734171204,
 1.6919729198037112e-06,
 0.

In [19]:
def getTopSim(similarities):
    #sim = similarities[0]
    simScores = list(enumerate(similarities))
    simScores = sorted(simScores, key=lambda x: x[1], reverse=True)
    simScores = simScores[:30]
    movieIdx = [i[0] for i in simScores]
    #return the title of the movie
    return originalDf['title'].iloc[movieIdx]
    #print(simScores)
    #sim[::-1].sort()
    #top10 = sim[:10]
    #return top10

top10 = getTopSim(resultSimilarities)
top10

9193                                 The End of Evangelion
13765           Neon Genesis Evangelion: Death and Rebirth
22166                   Evangelion: 3.0 You Can (Not) Redo
14054                 Evangelion: 1.0: You Are (Not) Alone
16632                Evangelion: 2.0 You Can (Not) Advance
37401                                                Asura
42505                 Saint Seiya Heaven Chapter: Overture
38187                                        Slayers Great
9425                       Ghost in the Shell 2: Innocence
27539                       Franz Kafka's a Country Doctor
35126                    Ghost in the Shell: The New Movie
23743    Gunbuster vs Diebuster Aim for the Top! The GA...
19279                      Dragon Ball GT: A Hero's Legacy
13030                                          Dead Leaves
12383                      The Girl Who Leapt Through Time
38188                                     Slayers Gorgeous
38177                                      Slayers Premi

In [26]:
originalDf.iloc[0:10]

,budget,genres,id,original_language,popularity,production_companies,release_date,runtime,title,cast,crew,director
0,30000000,"[Animation, Comedy, Family]",862,en,21.946943,[Pixar Animation Studios],1995-10-30,81.0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",John Lasseter
1,65000000,"[Adventure, Fantasy, Family]",8844,en,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",1995-12-15,104.0,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",Joe Johnston
2,0,"[Romance, Comedy]",15602,en,11.712900,"[Warner Bros., Lancaster Gate]",1995-12-22,101.0,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",Howard Deutch
3,16000000,"[Comedy, Drama, Romance]",31357,en,3.859495,[Twentieth Century Fox Film Corporation],1995-12-22,127.0,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devi...","[{'credit_id': '52fe44779251416c91011acb', 'de...",Forest Whitaker
4,0,[Comedy],11862,en,8.387519,"[Sandollar Productions, Touchstone Pictures]",1995-02-10,106.0,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",Charles Shyer
5,60000000,"[Action, Crime, Drama, Thriller]",949,en,17.924927,"[Regency Enterprises, Forward Pass, Warner Bros.]",1995-12-15,170.0,Heat,"[Al Pacino, Robert De Niro, Val Kilmer, Jon Vo...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",Michael Mann
6,58000000,"[Comedy, Romance]",11860,en,6.677277,"[Paramount Pictures, Scott Rudin Productions, ...",1995-12-15,127.0,Sabrina,"[Harrison Ford, Julia Ormond, Greg Kinnear, An...","[{'credit_id': '52fe44959251416c75039da9', 'de...",Sydney Pollack
7,0,"[Action, Adventure, Drama, Family]",45325,en,2.561161,[Walt Disney Pictures],1995-12-22,97.0,Tom and Huck,"[Jonathan Taylor Thomas, Brad Renfro, Rachael ...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",Peter Hewitt
8,35000000,"[Action, Adventure, Thriller]",9091,en,5.231580,"[Universal Pictures, Imperial Entertainment, S...",1995-12-22,106.0,Sudden Death,"[Jean-Claude Van Damme, Powers Boothe, Dorian ...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",Peter Hyams
9,58000000,"[Adventure, Action, Thriller]",710,en,14.686036,"[United Artists, Eon Productions]",1995-11-16,130.0,GoldenEye,"[Pierce Brosnan, Sean Bean, Izabella Scorupco,...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",Martin Campbell


In [22]:
originalDf.loc[23743]

budget                                                                  0
genres                                      [Animation, Adventure, Drama]
id                                                                  30146
original_language                                                      ja
popularity                                                        1.11678
production_companies    [Bandai Visual Company, Gainax, Victor Enterta...
release_date                                                   2006-10-01
runtime                                                               180
title                   Gunbuster vs Diebuster Aim for the Top! The GA...
cast                    [Noriko Hidaka, Norio Wakamoto, Maria Kawamura...
crew                    [{'credit_id': '53ce0f1bc3a368777a003f10', 'de...
director                                                     Hideaki Anno
Name: 23743, dtype: object

In [ ]:
sdf.to_csv('./csv/stringMeta.csv')

In [ ]:
sdf.dtypes